In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from keras import models
from keras import layers
import tensorflow as tf
from google.colab import files
import io
from sklearn.metrics import label_ranking_average_precision_score

# Model

In [3]:
uploaded = files.upload()

Saving 551_albums_audio.csv to 551_albums_audio.csv


In [0]:
audio_df = pd.read_csv(io.BytesIO(uploaded['551_albums_audio.csv']))

In [5]:
audio_df = audio_df.drop(columns = ['Unnamed: 0', 'index', 'amazonIDs'])
audio_df.head()

,Chroma Stft,Spectral Centroid,Spectral Bandwidth,Spectral Rolloff,ZCR,MFCC 1,MFCC 2,MFCC 3,MFCC 4,MFCC 5,Genres
0,0.317631,1858.848095,2070.823969,3804.451995,0.086750,-110.527584,107.819219,-12.992357,23.560247,0.107660,"Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
1,0.407539,2430.657788,1993.710689,4243.952115,0.150768,-152.042158,87.872922,-49.915526,10.911422,-15.193659,"Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
2,0.299010,1344.961601,1702.962194,2691.983722,0.059642,-135.593519,146.903281,-16.374589,33.878938,-6.721820,"Pop & Contemporary,Christian,Gospel,Pop, Pop &..."
3,0.344631,750.293388,1203.916434,1506.657556,0.031481,-287.927929,174.176362,5.348383,36.172204,39.698738,"Europe,World Music,Blues Rock,Rock,Pop,Dance &..."
4,0.209722,950.325207,1516.822655,1586.357074,0.043833,-206.839709,139.278957,1.349824,20.984522,16.714627,"New Age,Meditation,World Music,Pop, New Age,Me..."


In [0]:
scaler = MinMaxScaler()
X = scaler.fit_transform(np.array(audio_df.iloc[:, :-1], dtype = float))

In [0]:
def changeToTuple(x):
    stringListGenres = x.split(",")
    stringListGenres = tuple(stringListGenres)
    return(stringListGenres)

genres = audio_df['Genres']
genres = genres.apply(lambda x: changeToTuple(x))

genresList = genres.tolist()

mlb = MultiLabelBinarizer()

y = mlb.fit_transform(genresList)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [9]:
X_train.shape[1]

10

In [0]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(y_train.shape[1], activation='softmax'))

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(multi_label = True)])

In [12]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
440/440 [==============================] - 1s 2ms/step - loss: 0.1494 - accuracy: 0.9741 - auc: 0.3768
Epoch 2/20
440/440 [==============================] - 0s 915us/step - loss: 0.1388 - accuracy: 0.9741 - auc: 0.4719
Epoch 3/20
440/440 [==============================] - 0s 949us/step - loss: 0.1260 - accuracy: 0.9741 - auc: 0.4719
Epoch 4/20
440/440 [==============================] - 0s 929us/step - loss: 0.1226 - accuracy: 0.9741 - auc: 0.4722
Epoch 5/20
440/440 [==============================] - 0s 918us/step - loss: 0.1218 - accuracy: 0.9741 - auc: 0.4717
Epoch 6/20
440/440 [==============================] - 0s 919us/step - loss: 0.1215 - accuracy: 0.9741 - auc: 0.4719
Epoch 7/20
440/440 [==============================] - 0s 908us/step - loss: 0.1214 - accuracy: 0.9741 - auc: 0.4719
Epoch 8/20
440/440 [==============================] - 0s 957us/step - loss: 0.1212 - accuracy: 0.9741 - auc: 0.4719
Epoch 9/20
440/440 [==============================] - 0s 921us/step - loss

In [0]:
y_pred = model.predict(X_test)

In [15]:
LRAP = label_ranking_average_precision_score(y_test, y_pred)
print(LRAP)

0.42555410875277355
